In [18]:
import random

In [19]:
N = 8
POPULATION_SIZE = 100
GENERATIONS = 10000
MUTATION_RATE = 0.8
MAX_FITNESS = int((N * (N-1))/2)

In [20]:
# returns list of size N with all unique values
def generate_individual(N):
    individual = [x for x in range(N)]
    random.shuffle(individual)
    return individual

In [21]:
# test for generate_individual
for _ in range(5):
    print(generate_individual(N))

[6, 0, 5, 3, 7, 2, 1, 4]
[4, 6, 0, 5, 7, 1, 3, 2]
[0, 6, 1, 4, 5, 3, 7, 2]
[5, 3, 4, 6, 2, 0, 1, 7]
[1, 5, 2, 3, 7, 6, 4, 0]


In [22]:
# returns the fitness of a possible solution
# TODO: make this function work with arbitrary board sizes
def fitness(solution):
    attacking_queen_pairs = 0
    for i in range(N):
        for j in range(i+1, N):
            # if on same row or on diagonal
            if solution[i] == solution[j] or abs(solution[i] - solution[j]) == abs(i - j):
                attacking_queen_pairs += 1
                
    return MAX_FITNESS - attacking_queen_pairs

In [23]:
# fitness function tests
a = [7,1,4,2,0,6,3,5]
print(fitness(a))
b = [1,2,3,4,5,6,7,8]
print(fitness(b))

28
0


In [24]:
# mutates possible solution
def mutate(solution):
    if random.random() > (1 - MUTATION_RATE):
        idx_1 = random.randrange(N)
        idx_2 = random.randrange(N)
        # swap the two random indices
        solution[idx_1], solution[idx_2] = solution[idx_2], solution[idx_1]
        return solution
    else:
        return solution

In [25]:
# mutation test
a = [1,2,3,4,5,6,7,8]
print(mutate(a))

[1, 2, 3, 4, 5, 6, 7, 8]


In [26]:
# random one-point crossover function, returns two new children
def cut_and_crossover(solution1, solution2):
    cross_point = random.randrange(N) 
    # perform the crossover
    solution1[:cross_point], solution2[:cross_point] = solution2[:cross_point], solution1[:cross_point]
    return solution1, solution2

In [27]:
# tests of cut_and_crossover
a = [1,2,3,4,5,6,7,8]
b = [12,34,53,3,65,2,65,87]
print(cut_and_crossover(a,b))

([12, 34, 53, 3, 65, 2, 65, 8], [1, 2, 3, 4, 5, 6, 7, 87])


In [28]:
# finds and returns top two parents
def parent_selection(top_five):
    random.shuffle(top_five) # shuffle to ensure top solutions are not always chosen
    top_five = sorted(top_five, key=fitness, reverse=True) # sort by fitness
    return top_five[:2]

In [29]:
# tests for parent_selection and with cut_and_crossover
a = [0, 7, 2, 4, 6, 1, 3, 5]
b = [4, 1, 3, 0, 7, 5, 2, 6]
c = [3, 0, 6, 1, 5, 7, 2, 4]
d = [2, 4, 6, 1, 3, 0, 7, 5]
e = [3, 6, 2, 7, 0, 4, 5, 1]

top = [a,b,c,d,e]
parents = parent_selection(top)
print(f"{parents=}")
parent1, parent2 = cut_and_crossover(parents[0], parents[1])
print(f"{parent1=}, {parent2=}")

parents=[[0, 7, 2, 4, 6, 1, 3, 5], [3, 0, 6, 1, 5, 7, 2, 4]]
parent1=[3, 7, 2, 4, 6, 1, 3, 5], parent2=[0, 0, 6, 1, 5, 7, 2, 4]


In [30]:
# the evolutionary algorithm, returns the best solution and what generation it was found in
def evo_alg(N):
    # generate initial population
    population = [generate_individual(N) for _ in range(POPULATION_SIZE)]
    
    for generation in range(GENERATIONS):
        # sort population by best fitness to worst
        population = sorted(population, key=fitness, reverse=True)
        
        # early termination if true solution found
        if fitness(population[0]) == MAX_FITNESS:
            return population[0], generation
        
        # select top 5 parents, take best two and perform crossover for offspring
        parents = parent_selection(population[:5])
        child1, child2 = cut_and_crossover(parents[0], parents[1])
        
        # calculate fitness of children to then place into population
        child1_fitness = fitness(child1)
        child2_fitness = fitness(child2)
        
        # find and replace the first individual in population with lower fitness than child 1
        for idx, individual in enumerate(population):
            if child1_fitness > fitness(individual):
                population[idx] = child1
                break
                
        # find and replace the first individual in population with lower fitness than child 2
        for idx, individual in enumerate(population):
            if child2_fitness > fitness(individual):
                population[idx] = child2
                break
                
    return population[0], GENERATIONS

In [40]:
# run the evolutionary algorithm
sol, gen = evo_alg(N)
print(sol, gen)
print(fitness(sol))

[4, 6, 0, 2, 7, 5, 3, 1] 0
28


In [290]:
successful_attempts = 0
generations_of_successful_attempts = []
for i in range(100):
    sol, gen = evo_alg(N)
    if fitness(sol) == 28:
        successful_attempts += 1
        generations_of_successful_attempts.append(gen)
        
print(f"Evolutionary Algorithm Success Rate: {successful_attempts/100}")
print(generations_of_successful_attempts)
    

Evolutionary Algorithm Success Rate: 0.18
[0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
